In [10]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3
import numpy as np
from numpy import random
from time import time

In [11]:
#load data (make sure you have downloaded database.sqlite)
with sqlite3.connect('database.sqlite') as con:
    Player_Attributes = pd.read_sql_query("SELECT * from Player_Attributes", con)

In [12]:
#select relevant fields
Player_Attributes.dropna(inplace=True)
Player_Attributes.drop(['id', 'player_fifa_api_id', 'player_api_id', 'date'], axis = 1, inplace = True)
overall_rating = Player_Attributes['overall_rating']
features = Player_Attributes.drop('overall_rating', axis = 1)
features.head()

,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,71,right,medium,medium,49,44,71,61,44,51,...,54,48,65,69,69,6,11,10,8,8
1,71,right,medium,medium,49,44,71,61,44,51,...,54,48,65,69,69,6,11,10,8,8
2,66,right,medium,medium,49,44,71,61,44,51,...,54,48,65,66,69,6,11,10,8,8
3,65,right,medium,medium,48,43,70,60,43,50,...,53,47,62,63,66,5,10,9,7,7
4,65,right,medium,medium,48,43,70,60,43,50,...,53,47,62,63,66,5,10,9,7,7


1. Use encoding to convert catogarial values to numerical data.

2. Do feature scaling on GK traits.

3. Use PCA to short the features.

4. Use GridSearch and Boosting to tune the model

5. Use r2 to comapre accuracy.

In [13]:
# Use encoding to convert catogarial values to numerical data using LabelEncoder()
from sklearn import preprocessing

le_sex = preprocessing.LabelEncoder()

#to convert into numbers

features.preferred_foot = le_sex.fit_transform(features.preferred_foot)
features.attacking_work_rate = le_sex.fit_transform(features.attacking_work_rate)
features.defensive_work_rate = le_sex.fit_transform(features.defensive_work_rate)
features.head()

# to convert back
# train.Sex = le_sex.inverse_transform(train.Sex)

,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,71,1,4,14,49,44,71,61,44,51,...,54,48,65,69,69,6,11,10,8,8
1,71,1,4,14,49,44,71,61,44,51,...,54,48,65,69,69,6,11,10,8,8
2,66,1,4,14,49,44,71,61,44,51,...,54,48,65,66,69,6,11,10,8,8
3,65,1,4,14,48,43,70,60,43,50,...,53,47,62,63,66,5,10,9,7,7
4,65,1,4,14,48,43,70,60,43,50,...,53,47,62,63,66,5,10,9,7,7


In [14]:
list(features.columns)

['potential',
 'preferred_foot',
 'attacking_work_rate',
 'defensive_work_rate',
 'crossing',
 'finishing',
 'heading_accuracy',
 'short_passing',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'sprint_speed',
 'agility',
 'reactions',
 'balance',
 'shot_power',
 'jumping',
 'stamina',
 'strength',
 'long_shots',
 'aggression',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes']

In [16]:
# with sqlite3.connect('database.sqlite') as con:
#     Player_Attributes = pd.read_sql_query("SELECT curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning from Player_Attributes", con)
# Player_Attributes.head()

,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning
0,45,39,64,49,60,64,59,47,65,55,58,54,76,35,71,70,45
1,45,39,64,49,60,64,59,47,65,55,58,54,76,35,71,70,45
2,45,39,64,49,60,64,59,47,65,55,58,54,76,35,63,41,45
3,44,38,63,48,60,64,59,46,65,54,58,54,76,34,62,40,44
4,44,38,63,48,60,64,59,46,65,54,58,54,76,34,62,40,44


In [15]:
# Feature scaling using MinMaxScaler
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
scaled_features = min_max_scaler.fit_transform(features)
scaled_features

array([[ 0.55172414,  1.        ,  0.57142857, ...,  0.09375   ,
         0.07368421,  0.07368421],
       [ 0.55172414,  1.        ,  0.57142857, ...,  0.09375   ,
         0.07368421,  0.07368421],
       [ 0.46551724,  1.        ,  0.57142857, ...,  0.09375   ,
         0.07368421,  0.07368421],
       ..., 
       [ 0.70689655,  1.        ,  0.57142857, ...,  0.75      ,
         0.2       ,  0.2       ],
       [ 0.72413793,  1.        ,  0.57142857, ...,  0.75      ,
         0.2       ,  0.2       ],
       [ 0.72413793,  1.        ,  0.57142857, ...,  0.80208333,
         0.06315789,  0.14736842]])

In [6]:
# PCA to reduce feature reduction and to improve model speed
from sklearn.decomposition import PCA
pca = PCA(n_components = 6)
pca_features = pca.fit_transform(scaled_features)

In [7]:
# Train and predict model on Decision tree
from sklearn import tree
from sklearn.metrics import r2_score
from sklearn.cross_validation import train_test_split
from time import time

reg = tree.DecisionTreeClassifier()

t0 = time()
X_train, X_test, y_train, y_test = train_test_split(pca_features, overall_rating, test_size=0.40, random_state=0)
print X_train.shape
print "Time taken to split the data: {}".format(time()-t0)

t1 = time()
reg.fit(X_train, y_train)
print "Time taken to train the model: {}".format(time()-t1)

t2 = time()
pred = reg.predict(X_test)
print "Time taken to predict the model: {}".format(time()-t2)

t3 = time()
print "r2 score of this model is: {}".format(r2_score(y_test, pred))
print "Time taken to find the accuracy of model: {}".format(time()-t3)

(108212L, 6L)
Time taken to split the data: 0.269999980927
Time taken to train the model: 3.66799998283
Time taken to predict the model: 0.18499994278
r2 score of this model is: 0.920877052544
Time taken to find the accuracy of model: 0.0280001163483


In [8]:
# Use GridSearch to tune the model
def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    from sklearn.cross_validation import ShuffleSplit
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)

    # TODO: Create a decision tree regressor object
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.svm import SVC
    
    regressor1 = DecisionTreeRegressor()
    regressor2 = SVC()

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {'max_depth' : range(5, 20), 'min_samples_split':[2, 3, 4, 5]}
    tuned_parameters = {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]},{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}

    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    from sklearn.metrics import make_scorer
    scoring_fnc = make_scorer(performance_metric)

    # TODO: Create the grid search object
    from sklearn.grid_search import GridSearchCV
    
    # Updated cv_sets and scoring parameter
    grid = GridSearchCV(regressor1, params, scoring = scoring_fnc, cv = cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

def performance_metric(y_true, y_predict):
    
    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    from sklearn.metrics import r2_score
    
    score = r2_score(y_true, y_predict)
    # Return the score
    return score

In [9]:
from time import time
t0 = time()
grid_reg = fit_model(pca_features, overall_rating)
print grid_reg.score
# grid_pred = grid_reg()
print "Time taken to train and predict using GridSearch: {}".format(time() - t0)
print "Best parameters are: {}".format(grid_reg.get_params())

<bound method DecisionTreeRegressor.score of DecisionTreeRegressor(criterion='mse', max_depth=19, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=4,
           min_weight_fraction_leaf=0.0, random_state=None,
           splitter='best')>
Time taken to train and predict using GridSearch: 1499.69899988
Best parameters are: {'splitter': 'best', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 4, 'min_weight_fraction_leaf': 0.0, 'criterion': 'mse', 'random_state': None, 'max_features': None, 'max_depth': 19}
